In [81]:
import sys
print(sys.version)

3.6.8 (default, Oct 14 2019, 21:22:53) 
[GCC 4.8.5 20150623 (Red Hat 4.8.5-28)]


In [82]:
#!pip install seaborn
#!pip install matplotlib
!pip install plotly==4.6.0
!pip install sklearn
!pip install wordcloud

DEPRECATION: Python 2.7 reached the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 is no longer maintained. A future version of pip will drop support for Python 2.7. More details about Python 2 support in pip, can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support
Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: Python 2.7 reached the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 is no longer maintained. A future version of pip will drop support for Python 2.7. More details about Python 2 support in pip, can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support
Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: Python 2.7 reached the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 is no longer maintained. A future version of pip will drop su

In [43]:
from pyspark import SparkContext
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.functions import *
from pyspark.sql.types import DateType
from pyspark.sql.functions import countDistinct
import numpy as np
import pandas as pd 
import datetime

#import seaborn as sns
import string
import re
import nltk
import matplotlib.pyplot as plt

In [44]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
sql_sc = SQLContext(sc)

In [45]:
df= spark.read.format('csv').option("escape","\"").option('header',True).load("s3://msbx5420-2020/team_castle_peak/news_cleaned.csv/")
df.show(10)

+---+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+------------+------------------+-----------+----------+--------+
|_c0|             authors|               title|  publish_timestamp|         description|                text|                 url|title_length|description_length|text_length|      data|    time|
+---+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+------------+------------------+-----------+----------+--------+
|  1|        ['Cbc News']|Coronavirus a 'wa...|2020-03-27 08:00:00|Canadian pharmaci...|Canadian pharmaci...|https://www.cbc.c...|          66|               248|       2600|2020-03-27|08:00:00|
|  2|        ['Cbc News']|Yukon gov't names...|2020-03-27 01:45:00|The Yukon governm...|The Yukon governm...|https://www.cbc.c...|          60|               168|       1843|2020-03-27|01:45:00|
|  3|['The Associated ...

In [46]:
print(df.columns)

['_c0', 'authors', 'title', 'publish_timestamp', 'description', 'text', 'url', 'title_length', 'description_length', 'text_length', 'data', 'time']


In [47]:
df = df.drop('_c0','url','time','publish_timestamp')

In [48]:
from pyspark.sql.functions import *
def cleanColumn(tmpdf,colName,findChar,replaceChar):
    tmpdf = tmpdf.withColumn(colName, regexp_replace(colName, findChar, replaceChar))
    return tmpdf

allColNames = df.schema.names
charToRemove= "[\"!@#$%^&*\(\)\{\}\[\]\'\'""',.?/:;-=+`~'...''..']"
replaceWith =""
for colName in allColNames:
    df=cleanColumn(df,colName,charToRemove,replaceWith)

In [49]:
df.show(10)

+--------------------+--------------------+--------------------+--------------------+------------+------------------+-----------+----------+
|             authors|               title|         description|                text|title_length|description_length|text_length|      data|
+--------------------+--------------------+--------------------+--------------------+------------+------------------+-----------+----------+
|            Cbc News|Coronavirus a wak...|Canadian pharmaci...|Canadian pharmaci...|          66|               248|       2600|2020-03-27|
|            Cbc News|Yukon govt names ...|The Yukon governm...|The Yukon governm...|          60|               168|       1843|2020-03-27|
|The Associated Press|US Senate passes ...|The Senate has pa...|The Senate late W...|          49|               172|       6185|2020-03-26|
|            Cbc News|Coronavirus The l...|Scientists around...|Scientists around...|          65|               159|       5011|2020-03-27|
|            

In [50]:
df.createOrReplaceTempView("year")
df.createOrReplaceTempView("Author")

In [51]:
df_author = spark.sql("SELECT COUNT (DISTINCT authors) AS Author_Count FROM Author")
df_author.show()

+------------+
|Author_Count|
+------------+
|         261|
+------------+



In [52]:
df_year = spark.sql("SELECT COUNT (DISTINCT data) AS Author_Count FROM year where data LIKE '2020%'")
df_year.show()

+------------+
|Author_Count|
+------------+
|          78|
+------------+



In [53]:
df_year_2020 = spark.sql("SELECT data from year where data LIKE '2020%'")
df_year.show()

+------------+
|Author_Count|
+------------+
|          78|
+------------+



In [57]:
text_rdd = df.select('text').rdd.map(list)
splitted_word_rdd = text_rdd.flatMap(lambda line: line[0].split(' '))
print('Total word count:', splitted_word_rdd.count())
word_map_rdd = splitted_word_rdd.map(lambda word: (word, 1))
word_count_rdd = word_map_rdd.reduceByKey(lambda a, b: a + b)
print('Unique word count:', word_count_rdd.count())
word_count_list = word_count_rdd.collect()
print(word_count_list[0:10])

Total word count: 2012324
Unique word count: 80353
[('are', 16368), ('shortages', 152), ('recognizing', 16), ('active', 88), ('ingredients', 30), ('have', 13226), ('of', 47822), ('complying', 24), ('amount', 189), ('medications', 75)]


In [58]:
pdDF = pd.DataFrame(word_count_list)
pdDF.head(10)

,0,1
0,are,16368
1,shortages,152
2,recognizing,16
3,active,88
4,ingredients,30
5,have,13226
6,of,47822
7,complying,24
8,amount,189
9,medications,75


----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 37308)
Traceback (most recent call last):
  File "/usr/lib64/python3.6/socketserver.py", line 320, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib64/python3.6/socketserver.py", line 351, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib64/python3.6/socketserver.py", line 364, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib64/python3.6/socketserver.py", line 724, in __init__
    self.handle()
  File "/usr/lib/spark/python/pyspark/accumulators.py", line 269, in handle
    poll(accum_updates)
  File "/usr/lib/spark/python/pyspark/accumulators.py", line 241, in poll
    if func():
  File "/usr/lib/spark/python/pyspark/accumulators.py", line 245, in accum_updates
    num_updates = read_int(self.rfile)
  File "/usr/lib/spark/python/pyspark/serializers.py", lin